In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p ML
!google-drive-ocamlfuse ML
import os

os.chdir("ML/ML/datawhale/task4/") 

In [0]:
!pip install joblib
!pip install pandas
!pip install gbdt
!pip install xgboost
!pip install lightgbm
!pip install imbalanced-learn

    100% |████████████████████████████████| 3.1MB 10.6MB/s 
  Running setup.py bdist_wheel for gbdt ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/e1/4e/77/9269956665f9f92ebb16eec5029c5dc5ac98be513a3a649c1b
Successfully built gbdt


In [0]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler, scale
import lightgbm as lgb
import xgboost as xgb
import pandas as pd
import time
import matplotlib.pyplot as plt

In [6]:
data = pd.read_csv('../data_all.csv')
y_data = pd.DataFrame(data['status']).values.ravel()
x_data = data.drop(columns='status').values

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=2018)
print(data.shape)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(4754, 85)
(3327, 84) (3327,) (1427, 84) (1427,)


In [0]:
#数据归一化
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [0]:
'''
metric_func = {
                'accuracy_score':metrics.make_scorer(metrics.accuracy_score)
                ,'precision_score':metrics.make_scorer(metrics.precision_score)
                ,'recall_score':metrics.make_scorer(metrics.recall_score)
                ,'f1_score':metrics.make_scorer(metrics.f1_score)
                ,'roc_auc_score':metrics.make_scorer(metrics.roc_auc_score)
               }
'''

def get_optimizer(model, para_dict, file_path, x_train=x_train, y_train=y_train):
    kfold = KFold(n_splits=10)
    cv_result = {}
    #for i in metric_func.keys():
    grid = GridSearchCV(model, para, metrics.make_scorer(metrics.accuracy_score), cv=kfold, return_train_score=True)
    grid.fit(x_train, y_train)
    #print(i)
    print(grid.best_estimator_)
    print(grid.best_score_)
    pd_grid = pd.DataFrame(grid.cv_results_)
    pd_grid.to_csv(file_path)
    #cv_result[i] = grid.cv_results_

    #return cv_result
    return pd_grid
    

# LR

In [0]:
t_start = time.time()

lr = LogisticRegression(penalty='l1', class_weight='balanced', solver='liblinear', random_state=2018)

para={'C':[0.1, 0.3, 0.5, 0.7,0.9, 1],
      'tol':[1e-4, 1e-3, 1e-2, 1e-1],
      'max_iter':[100, 1000, 10000, 150000]
     }
lr_cvresult = get_optimizer(lr, para, 'lr_cvresult.csv')
t_end = time.time()
print('LR训练结束,耗时:{}min'.format((t_end - t_start) / 60))

LogisticRegression(C=0.1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=2018,
          solver='liblinear', tol=0.01, verbose=0, warm_start=False)
0.7460174331229336
LR训练结束,耗时:7.341310918331146min


# SVM

In [0]:
t_start = time.time()

svc = SVC(class_weight='balanced', gamma='scale', 
          random_state=2018)

para={'C':[0.1, 0.3, 0.5, 0.7, 0.9],
      'tol':[1e-3, 1e-4]
     }
svc_cvresult = get_optimizer(svc, para, 'svc_cvresult.csv')

#joblib.dump(svc, 'svc_model.m')

t_end = time.time()
print('SVM训练结束,耗时:{}min'.format((t_end - t_start) / 60))

SVC(C=0.5, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=2018, shrinking=True,
  tol=0.001, verbose=False)
0.7403065825067628
SVM训练结束,耗时:4.153508365154266min


# Tree

In [0]:
t_start = time.time()

dtree = DecisionTreeClassifier(random_state=2018, \
                               class_weight='balanced')

para = {'max_depth':range(10, 90, 10),\
               'min_samples_split':[0.1, 0.3, 0.5, 0.7, 0.9],
               'min_samples_leaf':range(1, 10, 1),\
               'min_weight_fraction_leaf':[0,.5],\
               'max_features':[0.1, 0.3, 0.5, 0.7, 0.9]
              }
dtree_cvresult = get_optimizer(dtree, para, 'dtree_cvresult.csv')

#joblib.dump(dtree, 'dtree_model.m')
t_end = time.time()
print('Tree训练结束,耗时:{}min'.format((t_end - t_start) / 60))

DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=10, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=0.7,
            min_weight_fraction_leaf=0, presort=False, random_state=2018,
            splitter='best')
0.7039374812143072
Tree训练结束,耗时:8.18286681175232min


# RF

In [0]:
t_start = time.time()

rf = RandomForestClassifier(class_weight='balanced', random_state=2018)

para = {'n_estimators':range(10,150,20),\
               'max_features':[0.1, 0.3, 0.5],\
               'max_depth':range(10,50,10),\
              }
rf_cvresult = get_optimizer(rf, para, 'rf_cvresult.csv')


#joblib.dump(rf, 'rf.m')

t_end = time.time()
print('rf 训练结束,耗时:{}min'.format((t_end - t_start) / 60))

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=10, max_features=0.1,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=110, n_jobs=None, oob_score=False,
            random_state=2018, verbose=0, warm_start=False)
0.795912233243162
rf 训练结束,耗时:25.84074740012487min


# GDBT

In [0]:
t_start = time.time()

gdbt = GradientBoostingClassifier(loss='exponential', random_state=2018)

param = {'n_estimators':[10,20],\
               'max_features':[0.1, 0.3, 0.5],\
               'max_depth':[10,20,30),\
              # 'min_samples_split':range(2,10,2),\
              # 'min_samples_leaf':range(2,10,2),\
              # 'max_leaf_nodes':range(10,50,10)
              }
gdbt_cvresult = get_optimizer(gdbt, para, 'gdbt_cvresult.csv')
#joblib.dump(gdbt, 'gdbt.m')

t_end = time.time()
print('gdbt 训练结束,耗时:{}min'.format((t_end - t_start) / 60))

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='exponential', max_depth=40,
              max_features=0.1, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=90,
              n_iter_no_change=None, presort='auto', random_state=2018,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)
0.7968139464983469
gdbt 训练结束,耗时:52.375156311194104min


# xgb

In [11]:
t_start = time.time()

xgbClassify = xgb.XGBClassifier(silent=1, booster='gbtree', 
                                     subsample=0.5,random_state=2018)

para={'min_child_weight':range(1,6,2),
      'gamma':[0.1,0.2]}

xgb_cvresult = get_optimizer(xgbClassify, para, 'xgb_cvresult.csv')

#joblib.dump(xgbClassify, 'xgbClassify_model.m')
t_end = time.time()
print('xgb训练结束,耗时:{}min'.format((t_end - t_start) / 60))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.1, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=5, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=2018, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=1, subsample=0.5)
0.7941088067327923
xgb训练结束,耗时:1.0993789394696554min


# lgb

In [12]:
t_start = time.time()

lgbClassify = lgb.LGBMClassifier()

para={'max_depth':range(3,9, 3),
      'n_estimators':[25,30,35],
      'criterion':["gini", "entropy"],
      'learning_rate':[0.1,0.4,0.5]}
lgb_cvresult = get_optimizer(lgbClassify,para, 'lgb_cvresult.csv')

#joblib.dump(lgbClassify, 'lgbClassify_model.m')
t_end = time.time()
print('lgb训练结束,耗时:{}min'.format((t_end - t_start) / 60))

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        criterion='gini', importance_type='split', learning_rate=0.1,
        max_depth=3, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=30, n_jobs=-1, num_leaves=31,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)
0.7968139464983469
lgb训练结束,耗时:1.1818571209907531min
